In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
import logging
from gensim.models import word2vec
from gensim.models import Word2Vec
import numpy as np

In [2]:
# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

#verify sizes
print("train: %d, test: %d, unlabeledtrain: %d" % (train['review'].size, test['review'].size, unlabeled_train['review'].size))

train: 25000, test: 25000, unlabeledtrain: 50000


In [3]:
def review_to_wordlist(raw_review, remove_stopwords=False):
    #remove html <- beautiful soup
    review_text = BeautifulSoup(raw_review).get_text()
    
    #remove bad chars <- re
    only_letters = re.sub("[^A-Za-z]", " ", review_text)
    word_list = only_letters.lower().split()
    
    #optionally remove stopwords and return as list
    if remove_stopwords:
        stop_words = set(stopwords.words("english"))
        word_list = [w for w in word_list if not w in stop_words]
        
    return word_list   

In [4]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    #tokenize into sentences based off of nltk punkt
    raw_sentences = tokenizer.tokenize(review.strip())
    
    sentences = []
    
    #clean each sentence
    for s in raw_sentences:
        if len(s) > 0:
            sentences.append(review_to_wordlist(s, remove_stopwords))
    
    return sentences #list of lists

In [5]:
sentences = []

print("Parsing labelled training set...")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print(sentences[0])
print(sentences[1])
print(len(sentences))

print("Parsing unlabelled training set...")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing labelled training set...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']
266551
Parsing labelled training set...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL.

In [6]:
print(len(sentences))

795538


In [7]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [8]:
# Set values for word2vec parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the word2vec model (takes time)
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# save the model for later use. Load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-09-03 12:28:22,792 : INFO : collecting all words and their counts
2018-09-03 12:28:22,793 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-09-03 12:28:22,835 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-09-03 12:28:22,877 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-09-03 12:28:22,916 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-09-03 12:28:22,963 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types


Training model...


2018-09-03 12:28:23,012 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-09-03 12:28:23,057 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-09-03 12:28:23,104 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-09-03 12:28:23,151 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-09-03 12:28:23,196 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-09-03 12:28:23,243 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2018-09-03 12:28:23,290 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2018-09-03 12:28:23,333 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keeping 54119 word types
2018-09-03 12:28:23,376 : INFO : PROGRESS: at sentence #130000, processed 2894303 words, keep

2018-09-03 12:28:26,387 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2018-09-03 12:28:26,438 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2018-09-03 12:28:26,483 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2018-09-03 12:28:26,515 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2018-09-03 12:28:26,516 : INFO : Loading a fresh vocabulary
2018-09-03 12:28:26,601 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2018-09-03 12:28:26,601 : INFO : effective_min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2018-09-03 12:28:26,642 : INFO : deleting the raw counts dictionary of 123504 items
2018-09-03 12:28:26,646 : INFO : sample=0.001 downsamples 48 most-common words
2018-09-03 12:28:26,647 : INFO : downsampling leaves estimated 1

2018-09-03 12:29:17,678 : INFO : EPOCH 4 - PROGRESS: at 73.17% examples, 928379 words/s, in_qsize 7, out_qsize 0
2018-09-03 12:29:18,681 : INFO : EPOCH 4 - PROGRESS: at 80.53% examples, 928895 words/s, in_qsize 7, out_qsize 0
2018-09-03 12:29:19,691 : INFO : EPOCH 4 - PROGRESS: at 87.59% examples, 925874 words/s, in_qsize 7, out_qsize 0
2018-09-03 12:29:20,701 : INFO : EPOCH 4 - PROGRESS: at 95.00% examples, 926628 words/s, in_qsize 7, out_qsize 0
2018-09-03 12:29:21,355 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-03 12:29:21,365 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-03 12:29:21,370 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-03 12:29:21,374 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-03 12:29:21,375 : INFO : EPOCH - 4 : training on 17798082 raw words (12750232 effective words) took 13.7s, 927966 effective words/s
2018-09-03 12:29:22,393 : INFO : EPOCH 5 - P

In [9]:
model.most_similar("raven".split())

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('dahlia', 0.5981990098953247),
 ('buxom', 0.5907309055328369),
 ('foxy', 0.5887882709503174),
 ('butch', 0.5813795328140259),
 ('sheep', 0.573807954788208),
 ('gray', 0.5734368562698364),
 ('stallion', 0.5704995393753052),
 ('regina', 0.5704531669616699),
 ('bald', 0.5673142671585083),
 ('moreland', 0.5661356449127197)]

In [10]:
model = Word2Vec.load("300features_40minwords_10context")

2018-09-03 12:29:38,656 : INFO : loading Word2Vec object from 300features_40minwords_10context
2018-09-03 12:29:39,027 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2018-09-03 12:29:39,028 : INFO : setting ignored attribute vectors_norm to None
2018-09-03 12:29:39,029 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2018-09-03 12:29:39,029 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2018-09-03 12:29:39,030 : INFO : setting ignored attribute cum_table to None
2018-09-03 12:29:39,031 : INFO : loaded 300features_40minwords_10context


In [16]:
# Function to average all of the word vectors in a given paragraph
def makeFeatureVec(words, model, num_features):
    #make a vector of size num_features 
    #note for numpy: if the second shape val is not listed, then it's a vector, not a matrix
    featureVec = np.zeros((num_features,),dtype="float32")
    
    nwords = 0.
    # Index2word is a list that contains the names of the words in the model's vocabulary
    index2word_set = set(model.wv.index2word)

    # Loop over each word in the review and, if it is in the model's vocab, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
            
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    
    return featureVec

In [19]:
def getAvgFeatureVecs(reviews, model, num_features):
    # calculate the average feature vector for each review and return a 2D numpy array 
    
    # Initialize a counter
    counter = 0
    
    # pre-allocate a 2d array with len(reviews) rows and num_features columns
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    # loop through the reviews
    for review in reviews:
        if counter%1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
       
        #make avg feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
    
       counter = counter + 1
        
    return reviewFeatureVecs

In [20]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [21]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [27]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] // 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


Time taken for K Means clustering:  382.23530983924866 seconds.


In [29]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [32]:
# For the first 10 clusters
for cluster in range(10):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in word_centroid_map.keys():
        if( word_centroid_map[i] == cluster ):
            words.append(i)
    print(words)


Cluster 0
['forget', 'relate', 'argue', 'resist', 'deny', 'tolerate']

Cluster 1
['nutty', 'sawyer']

Cluster 2
['bums']

Cluster 3
['blandings', 'perfected', 'moonstruck', 'inimitable']

Cluster 4
['scotland']

Cluster 5
['lola', 'dallas', 'herman', 'logan', 'maine', 'louie', 'hastings']

Cluster 6
['false']

Cluster 7
['dangers', 'evils', 'rapture', 'statistics']

Cluster 8
['joined', 'spotted', 'freed', 'proposed', 'persuaded', 'summoned', 'defended', 'parliament']

Cluster 9
['unexciting', 'uninvolving', 'compact']


In [33]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    # The number of clusters is equal to the highest cluster index in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1

    # Pre-allocate the bag of centroids vector as a vector with num features = num of centroids
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    
    # Return the "bag of centroids"
    return bag_of_centroids

In [34]:
# Pre-allocate a matrix with rows = num of training reviews and cols = number of clusters/features
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [36]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest to training set, may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])

#predicting test results
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
